In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import animation, patches, cm, rc
from matplotlib import pyplot as plt
from statsmodels.api import Logit
from Tomato3_dataStructure import Trial, Subject, Experiment
from helper import angle_overtake, lateral_overtake, overtake_rates, max_freewalk_spd, average_freewalk_spd,\
                    average_onset_spds, expansion, expansion_at, average_expansions, running_average,\
                    overtake_onset
%matplotlib qt
rc('font', size=14)

'''import trials'''
Hz = 90
exp = Experiment()
for i in range(1,13):
    exp.subjects[i] = Subject(i)
    if i%2 == 0:
        exp.subjects[i].leader = 'avatar'
    else:
        exp.subjects[i].leader = 'pole'
    
input_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Tomato3_rawData\Tomato3_input'))
output_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Tomato3_rawData\Tomato3_output'))

for output_file in os.listdir(output_dir):
    output_file_path = os.path.join(output_dir, output_file)
    # import experimental data
    if  'Tomato3_subj' in output_file and '.csv' in output_file:
        with open(output_file_path, 'r') as f:
            df = pd.read_csv(f, header=None)
            lpos = np.array(df.iloc[:, [0,2,1]])
            fpos = np.array(df.iloc[:, [3,5,4]])
            fori = np.array(df.iloc[:, 6:9])
            tstamps = np.array(df.iloc[:, 9])
            leader_model = np.array(df.iloc[:, 10])
            subject_id = int(output_file[12:14])
            trial_id = int(output_file[20:23])
            v0 = float(output_file[27:30])
            if output_file[-5] == 'e':
                leader = 'pole'
            elif output_file[-5] == 'r':
                leader = 'avatar'
            leader_onset = None
            
            exp.subjects[subject_id].trials[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                tstamps, v0, leader, leader_onset, leader_model)           
    # import IPD and gender data
    elif 'IPD' in output_file:    
        with open(output_file_path, 'r') as f:
            subject_id = int(output_file[12:14])
            exp.subjects[subject_id].gender == output_file[19]
            i = output_file.find('txt')
            exp.subjects[subject_id].IPD == float(output_file[20:i-1])
    # import freewalk data
    elif 'freewalk' in output_file:
        with open(output_file_path, 'r') as f:
            df = pd.read_csv(f, header=None)
            subject_id = int(output_file[21:23])
            session = int(output_file[-14])
            trial_id = int(output_file[-7:-4])
            v0 = 0
            leader = leader_onset = leader_model= None
            fpos = np.array(df.iloc[:,[0,2,1]])
            lpos = np.tile([0,0,0], (len(fpos), 1))
            fori = np.array(df.iloc[:,3:6])
            tstamps = np.array(df.iloc[:,-1])
            if session == 1:
                exp.subjects[subject_id].freewalk[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                    tstamps, v0, leader, leader_onset, leader_model)
            else:
                trial_id += 4
                exp.subjects[subject_id].freewalk[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                    tstamps, v0, leader, leader_onset, leader_model)
# import inputs
for input_file in os.listdir(input_dir):
    if 'Tomato3_subject' in input_file:
        subject_id = int(input_file[-6:-4])
        if subject_id in exp.subjects and exp.subjects[subject_id].trials != {}:
            # read experimental trials
            with open(os.path.join(input_dir, input_file), 'r') as f:
                df = pd.read_csv(f)
                for i in range(len(df)):
                    trial_id = df.iloc[i,0]
                    leader_onset = df.iloc[i,4]
                    exp.subjects[subject_id].trials[trial_id].leader_onset = leader_onset
 

# with open(filename, 'r') as file:
#     rows = file.read().split('\n')[:-1]
#     cells = [row.split(',') for row in rows]  
#     cells = np.array([[float(s) for s in row] for row in cells])
#     lpos = cells[:,[0,2,1]]
#     fpos = cells[:,[3,5,4]]
#     fori = cells[:,6:9]
#     tstamps = cells[:,-1]



In [26]:
'''debug'''
subject = 12
trial = 38
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        deviation = t.get_positions('f')[t.f1, 0]
#         if abs(deviation) >= 0.2:
#             print('subject ', s.id, ' trial ', t.trial_id, ' deviation ', round(deviation, 2))


In [2]:
'''Plot positions, speed, or acceleration of freewalk trials'''
subject = 3
for i, t in exp.subjects[subject].freewalk.items():
    t.plot_speeds(distance=False)
#     t.plot_trajectory('f')
#     t.plot_accelerations()


In [2]:
'''plot trajectories, speeds, or accelerations of experimental trials'''
%matplotlib qt
frames = None
subject = [2]
trials = range(1, 61)
filtered = True
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i in subject and j in trials:
            if t.v0 in [1.2] and not lateral_overtake(t, 0.3):
#                 frames = list(range(t.f1, len(t.tstamps)))
#                 t.plot_trajectory(frames=frames, accelerations=False, links=True, filtered=True)
                t.plot_speeds(frames=frames, component='y', filtered=filtered)
#                 t.plot_accelerations(frames=frames, component='')
                label = 'overtake' if lateral_overtake(t, 0.3) else ''
                plt.text(0, 0, label) # overtake label

In [132]:
'''plot lateral speeds of experimental trials'''
%matplotlib qt
frames = None
subject = [1]
trials = range(1,61)
filtered=True
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i in subject and j in trials:
            if t.v0 in [1.1] and lateral_overtake(t, 0.3):
#                 frames = list(range(t.f1, len(t.tstamps)))
                t.plot_speeds(frames=frames, component='x', filtered=filtered)
                label = 'overtake' if lateral_overtake(t, 0.3) else ''
                plt.text(0, 0, label) # overtake label
                plt.plot(t.tstamps_smooth, running_average(t, 'vel')[:, 0], 'g') # running average
                plt.plot([0, 12], [0, 0], 'r') # zero line
                plt.plot([t.tstamps_smooth[t.f1], t.tstamps_smooth[t.f1]], [-1, 1], 'k') # leader appear
                o = overtake_onset(t, tolerance=0.02)
                fvel_x = t.get_velocities('f')[:, 0]
                plt.scatter(t.get_time(filtered)[o], fvel_x[o], edgecolors='r', facecolors='none') # onset of overtaking

In [29]:
'''plot lateral positions of experimental trials'''
%matplotlib qt
frames = None
subject = [12]
trials = range(1, 61)
filtered=True
for i, s in exp.subjects.items():
    for j, t in s.trials.items():
        if i in subject and j in trials:
            if t.v0 in [1.2, 1.3] and lateral_overtake(t, 0.3):
#                 frames = list(range(t.f1, len(t.tstamps)))
                t.plot_positions(frames=frames, component='x', filtered=filtered)
                label = 'overtake' if lateral_overtake(t, 0.3) else ''
                plt.text(0, 0, label) # overtake label
                plt.plot([0, 12], [0, 0], 'r') # zero line
                plt.plot([t.tstamps_smooth[t.f1], t.tstamps_smooth[t.f1]], [-2, 2], 'k') # leader appear
                o = overtake_onset(t, tolerance=0.02)
                fpos_x = t.get_positions('f')[:, 0]
                plt.scatter(t.get_time(filtered)[o], fpos_x[o], edgecolors='r', facecolors='none') # onset of overtaking

In [131]:
'''animation'''
# compare trial 6 (catch up) and 5 (let it go)
subject = 1
trial = 41
exp.subjects[subject].trials[trial].play_trial(save=False, filtered=False)

In [5]:
'''lateral deviation in freewalk trials'''
max_deviations = {}
for i, s in exp.subjects.items():
    max_deviation = 0
    for j, t in s.freewalk.items():
        _max = max(abs(t.get_positions('f')[:, 0]))
        if _max > max_deviation:
            max_deviation = _max
    max_deviations[i] = round(max_deviation, 3)
print(max_deviations)

{1: 0.102, 2: 0.097, 3: 0.207, 4: 0.205, 5: 0.281, 6: 0.144, 7: 0.121, 8: 0.148, 9: 0.394, 10: 0.304, 11: 0.128, 12: 0.12}


In [18]:
'''
    Overtake rate by v0 by subject (12 plots). Overtaking is defined by angle 
    and lateral deviation criteria.
'''
for i, s in exp.subjects.items():
    ot_angle = {0.8:0, 0.9:0, 1.0:0, 1.1:0, 1.2:0, 1.3:0}
    ot_lateral = {0.8:0, 0.9:0, 1.0:0, 1.1:0, 1.2:0, 1.3:0}

    for j, t in s.trials.items():
        if angle_overtake(t, 55):
            ot_angle[t.v0] += 0.1
        if lateral_overtake(t, 0.3):
            ot_lateral[t.v0] += 0.1
    fig = plt.figure()
    ax = plt.axes(xlim=(0.7, 1.4), ylim=(-0.1, 1.1))
    ax.plot(list(ot_angle.keys()),list(ot_angle.values()))
    ax.plot(list(ot_lateral.keys()),list(ot_lateral.values()))
    plt.title('subject ' + str(i))

In [41]:
'''Histograms'''
##########control panel###############
var = 'dv' # 'dv' speed difference, 'exp' expansion rate, 'dist' distance, 'ttp' time to pass
scaler = 'max' # 'max', 'mean' or 'onset'
######################################

for i, s in exp.subjects.items():
    vars = []
    for j, t in s.trials.items():
        if lateral_overtake(t, 0.3):
            if var == 'dv':
                fspd = t.get_speeds('f')[t.f1]
                vars.append(fspd-t.v0)
    fig = plt.figure()
    ax = plt.axes()
    ax.hist(vars, 5)

In [5]:
'''Compare Overtake rate on pole and avatar'''

##########control panel###############
x_value = 'ratio' # 'ratio', 'diff' or 'exp'
p_fspd_type = 'max' # 'max', 'mean' or 'onset'
window = 2 # time window used for compute the mean speed
######################################
# set the range of x axis
if x_value == 'ratio': xrange = (0.4, 1.2)
elif x_value == 'diff': xrange = (-0.2, 0.8)
elif x_value == 'exp': xrange = (0.1, 0.5)


# build figure for all in one plot
fig_all = plt.figure()
ax = plt.axes(xlim=xrange, ylim=(-0.1, 1.1))
ax.plot(xrange,[0.5, 0.5], 'k')
ax.set_ylabel('Probability of overtaking')
ax.set_xlabel(x_value + ' between ' + p_fspd_type + ' follower speed and leader speed')
ax.set_title('All subjects')

for i, s in exp.subjects.items():
    
    # get speed scaler
    if p_fspd_type == 'max':
        p_fspd = max_freewalk_spd(s)
        p_fspd = [p_fspd] * 6
    elif p_fspd_type == 'mean':
        p_fspd = average_freewalk_spd(s, window)
        p_fspd = [p_fspd] * 6
    elif p_fspd_type == 'onset':
        p_fspd = average_onset_spds(s)
    
    # compute overtake rate
    rate = overtake_rates(s, 0.3)
    
    # prepare x values and line color
    color = 'b' if s.leader == 'pole' else 'r'
    if x_value == 'ratio':
        x = [a / 10.0 / b for a, b in zip(range(8, 14), p_fspd)]
    elif x_value == 'diff':
        x = [b - a / 10.0 for a, b in zip(range(8, 14), p_fspd)]
    elif x_value == 'exp':
        x = average_expansions(s, relative=False)
        
    # plot overtake rate by scaled speed on all in one plot
    if s.leader == 'pole':
        pole = fig_all.axes[0].plot(x, list(rate.values()), c=color)
    elif s.leader == 'avatar':
        avatar = fig_all.axes[0].plot(x, list(rate.values()), c=color)
    
    # build figure for separate plots
    fig = plt.figure()
    ax = plt.axes(xlim=xrange, ylim=(-0.1, 1.1))
    ax.set_ylabel('Probability of overtaking')
    ax.set_xlabel(x_value + ' between ' + p_fspd_type + ' follower speed and leader speed')
    ax.set_title('subject ' + str(i))
    
    # plot overtake rate by scaled speed on separate plot
    ax.plot(x, list(rate.values()))
    ax.plot(xrange, [0.5, 0.5], 'k')
    
fig_all.axes[0].legend((pole[0],avatar[0]), ('pole', 'avatar'))

In [8]:
'''
    Compare overtaking and following trials on fspd and fspd/max_fspd 
    when leader appear.
'''
########Control Panel######
relative = True
difference = True
##########################


data = {0.8:[[],[]], 0.9:[[],[]], 1.0:[[],[]], 1.1:[[],[]], 1.2:[[],[]], 1.3:[[],[]]}

for i, s in exp.subjects.items():
    max_fspd = max_freewalk_spd(s)
    for j, t in s.trials.items():
        fspd = t.get_speeds('f')[t.f1]
        if difference:
            fspd -= t.v0
        if relative:
            fspd /= max_fspd
        if lateral_overtake(t, 0.3):
            data[t.v0][1].append(fspd)
        else:
            data[t.v0][0].append(fspd)
            
for v0, fspd in data.items():
    # creat figure
    fig = plt.figure()
    if relative and difference:
        ax = plt.axes(ylim=(-0.5, 0.6))
    elif relative:
        ax = plt.axes(ylim=(0.3, 1.2))
    elif difference:
        ax = plt.axes(ylim=(-0.5, 0.9))
    else:
        ax = plt.axes(ylim=(0.5, 1.7))
    ax.set_title('leader speed = {}'.format(v0))
    ax.set_xticklabels(['follow', 'overtake'])
    ax.set_ylabel('follower speed when leader appears')
    
    # make boxplot
    ax.boxplot(fspd)
    
    # mark leader speed
    if not relative and not difference:
        ax.plot([0.5, 2.5], [v0, v0])

    # add scatter points on top of boxplot
    for i in [1,2]:
        x = np.random.normal(i, 0.04, size=len(fspd[i-1]))
        ax.plot(x, fspd[i-1], 'r.', alpha=0.2)


In [117]:
from statsmodels.regression.mixed_linear_model import MixedLM

'''Logistic regression---------load data'''

# data when leader appears
data = []
for i, s in exp.subjects.items():
    fmax = max_freewalk_spd(s)
    fmean = average_freewalk_spd(s, 2)
    for j, t in s.trials.items():
        lspd = t.v0
        fspd = t.get_speeds('f')[t.f1]
        data.append([i, 
                     lspd, fspd, fmax, fmean,
                     lspd / fspd, lspd / fmax, lspd / fmean,
                     fspd / fmax, fspd / fmean,
                     lspd * fspd, lspd * fmax, lspd * fmean,
                     fspd * fmax, fspd * fmean,
                     lspd * fspd * fmax,
                     int(lateral_overtake(t, 0.3))])

df = pd.DataFrame(data, columns=['subj', 
                                 'lspd', 'fspd', 'fmax', 'fmean',
                                 'lspd/fspd', 'lspd/fmax', 'lspd/fmean',
                                 'fspd/fmax', 'fspd/fmean',
                                 'lspd*fspd', 'lspd*fmax', 'lspd*fmean',
                                 'fspd*fmax', 'fspd*fmean',
                                 'lspd*fspd*fmax',
                                 'overtakeing'])
df

# data at the onset of overtaking


,subj,lspd,fspd,fmax,fmean,lspd/fspd,lspd/fmax,lspd/fmean,fspd/fmax,fspd/fmean,lspd*fspd,lspd*fmax,lspd*fmean,fspd*fmax,fspd*fmean,lspd*fspd*fmax,overtakeing
0,1,1.0,0.971135,1.420431,1.327614,1.029723,0.704012,0.753231,0.683690,0.731489,0.971135,1.420431,1.327614,1.379429,1.289292,1.379429,0
1,1,0.8,1.089544,1.420431,1.327614,0.734252,0.563209,0.602585,0.767052,0.820679,0.871636,1.136344,1.062091,1.547622,1.446495,1.238098,1
2,1,0.8,1.322598,1.420431,1.327614,0.604870,0.563209,0.602585,0.931125,0.996222,1.058078,1.136344,1.062091,1.878658,1.755900,1.502927,1
3,1,1.2,1.246889,1.420431,1.327614,0.962395,0.844814,0.903877,0.877824,0.939195,1.496267,1.704517,1.593137,1.771119,1.655387,2.125343,0
4,1,1.3,1.329659,1.420431,1.327614,0.977694,0.915215,0.979200,0.936096,1.001540,1.728557,1.846560,1.725898,1.888688,1.765274,2.455295,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,12,0.9,1.331696,1.443754,1.378456,0.675830,0.623375,0.652904,0.922385,0.966078,1.198527,1.299378,1.240611,1.922642,1.835685,1.730378,1
716,12,1.2,1.344588,1.443754,1.378456,0.892466,0.831167,0.870539,0.931314,0.975430,1.613506,1.732504,1.654148,1.941254,1.853456,2.329505,1
717,12,1.1,1.346209,1.443754,1.378456,0.817109,0.761903,0.797994,0.932437,0.976606,1.480830,1.588129,1.516302,1.943594,1.855690,2.137954,1
718,12,0.8,1.388641,1.443754,1.378456,0.576103,0.554111,0.580359,0.961827,1.007388,1.110913,1.155003,1.102765,2.004856,1.914181,1.603885,1


In [122]:
'''Logistic regression'''

lr = Logit(df['overtakeing'], df[['fspd']])
lr_results = lr.fit()
print(lr_results.summary2())

lr = Logit(df['overtakeing'], df[['lspd', 'fspd']])
lr_results = lr.fit()
print(lr_results.summary2())

lr = Logit(df['overtakeing'], df[['lspd', 'fspd', 'lspd*fspd*fmax']])
lr_results = lr.fit()
print(lr_results.summary2())


Optimization terminated successfully.
         Current function value: 0.674475
         Iterations 4
                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.007   
Dependent Variable: overtakeing      AIC:              973.2439
Date:               2019-11-15 12:54 BIC:              977.8232
No. Observations:   720              Log-Likelihood:   -485.62 
Df Model:           0                LL-Null:          -489.02 
Df Residuals:       719              LLR p-value:      nan     
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     4.0000                                     
------------------------------------------------------------------
        Coef.     Std.Err.      z       P>|z|     [0.025    0.975]
------------------------------------------------------------------
fspd    0.3092      0.0602    5.1338    0.0000    0.1912    0.4273

Optimization terminated successfully.
         Current function value: 0.34375

In [127]:
'''Linear mixed effect'''

lme = MixedLM(df['overtakeing'], df[['lspd', 'fspd', 'lspd/fspd', 'fmax']], groups=df['subj'])
lme_results = lme.fit()
print(lme_results.summary())

lme = MixedLM(df['overtakeing'], df[['lspd', 'fspd', 'lspd/fspd']], groups=df['subj'])
lme_results = lme.fit()
print(lme_results.summary())

lme = MixedLM(df['fspd'], df[['fmax']], groups=df['subj'])
lme_results = lme.fit()
print(lme_results.summary())


          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: overtakeing
No. Observations: 720     Method:             REML       
No. Groups:       12      Scale:              0.0883     
Min. group size:  60      Likelihood:         -167.7893  
Max. group size:  60      Converged:          Yes        
Mean group size:  60.0                                   
----------------------------------------------------------
           Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------
lspd       -2.058     0.241  -8.529  0.000  -2.531  -1.585
fspd        0.477     0.292   1.631  0.103  -0.096   1.050
lspd/fspd  -0.074     0.284  -0.262  0.793  -0.631   0.482
fmax        1.511     0.257   5.890  0.000   1.008   2.014
Group Var   0.020     0.032                               

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: overtakeing
No. Observations: 720    